In [ ]:
import os
import numpy as np
import pandas as pd
from fancyimpute import KNN 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
os.chdir("D:\Edwisor project\Employee Absenteeism")

In [ ]:
xls=pd.ExcelFile("Absenteeism_at_work_Project.xls")

In [ ]:
df=xls.parse("Absenteeism_at_work")

In [ ]:
df.info()

# Missing Value analysis

In [ ]:
# Create a dataframe with missing value counf for each variable
miss_val=pd.DataFrame(df.isnull().sum()).reset_index()

# Renaming the variables of columns in the dataframe
miss_val=miss_val.rename(columns={'index':'Variable',0:'Missing_Percentage'})

miss_val['Missing_Percentage']=(miss_val['Missing_Percentage']/len(df))*100
# Soet the value
miss_val=miss_val.sort_values('Missing_Percentage',ascending=False).reset_index(drop=True)

In [ ]:
miss_val

# Imputation method

In [ ]:
df=df.drop(df[df['Absenteeism time in hours'].isnull()].index,axis=0)
df.shape

In [ ]:
# 31.0
#26.70 mean
# 25 median
# 30.99
df["Body mass index"].iloc[12]=np.nan

In [ ]:
# Impute missing value with mea,mediuan and KNN method
df["Body mass index"]=df["Body mass index"].fillna(df["Body mass index"].mean())

In [ ]:
df["Body mass index"]=df["Body mass index"].fillna(df["Body mass index"].median())

In [ ]:
df=pd.DataFrame(KNN(k=3).complete(df),columns = df.columns)

In [ ]:
df["Body mass index"].iloc[12]

In [ ]:
# Convert the variable to category
categorical_vars = ['ID','Reason for absence','Month of absence','Day of the week',
                     'Seasons','Disciplinary failure', 'Education', 'Social drinker',
                     'Social smoker', 'Pet', 'Son']
for var in categorical_vars:
    df[var]=df[var].astype('int').astype('object',copy=False)

In [ ]:
df2=df.copy()

In [ ]:
# Box plot o visualize the number of ouliers
num_df=df._get_numeric_data()
for i in num_df.columns:
     print(i)
     if(i !="Absenteeism time in hours"):
        df.boxplot(column=i,by='Absenteeism time in hours')

In [ ]:
import seaborn as sns

In [ ]:
df.isnull().sum().sum()

# Feature Selection 


In [ ]:
import seaborn as sns
##Correlation analysis for continuous variables
#Correlation plot
f,ax=plt.subplots(figsize=[10,8])
corr_data=num_df.corr()
sns.heatmap(corr_data,mask=np.zeros_like(corr_data,dtype=np.bool),cmap=sns.diverging_palette(220,10,as_cmap=True),
           square=True,ax=ax,annot = True)


In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
# Convert the variable to category
categorical_vars = ['ID','Reason_for_absence','Month_of_absence','Day_of_the_week',
                     'Seasons','Disciplinary_failure', 'Education', 'Social_drinker',
                     'Social_smoker', 'Pet', 'Son']
for var in categorical_vars:
    df[var]=df[var].astype('int').astype('object',copy=False)

In [ ]:
num_df=df._get_numeric_data()
cols=list(set(df.columns) - set(num_df.columns))

In [ ]:
num_df.columns

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
for col in cols:
    results = ols('Absenteeism_time_in_hours~'+col, data=df).fit()
    print(sm.stats.anova_lm(results, typ=2))

In [ ]:
df=df.drop(["ID","Age","Month_of_absence","Disciplinary_failure","Seasons","Education","Pet","Social_smoker","Social_drinker","Weight","Height"],axis = 1)

In [ ]:
df.shape

# Feature Scaling

In [ ]:
# Code to plot the histogram for all the numeric variables
num_df=df._get_numeric_data()
for i, col in enumerate(num_df.columns):
    if col != "Absenteeism time in hours":
        plt.figure(i)
        sns.distplot(num_df[col])
        plt.title("Checking Distribution for Variable "+col)

In [ ]:
# Since there is no normally distributed curve we will use Normalizationg for Feature Scalling
# #Normalization
for i in num_df.columns:
    print(i)
    df[i]=(df[i]-min(df[i]))/(max(df[i])-min(df[i]))

# Data Modeling

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
train,test = train_test_split( df, test_size = 0.20)

In [ ]:
# Building model on top of training dataset
fit_DT = DecisionTreeRegressor(max_depth=3).fit(train.iloc[:,0:9],train.iloc[:,9])

# Calculating RMSE for test data to check accuracy
pred_test = fit_DT.predict(test.iloc[:,0:9])
rmse_for_test =np.sqrt(mean_squared_error(test.iloc[:,9],pred_test))
print("Root Mean Squared Error For Test data = "+str(rmse_for_test))
print("R^2 Score(coefficient of determination) = "+str(r2_score(test.iloc[:,9],pred_test)))

In [ ]:
df.dtypes

# Linear Regession

In [ ]:
LRMod=sm.OLS(train.iloc[:,9],(train.iloc[:,0:9]).astype(float)).fit()
LRMod.summary()

In [ ]:
# Calculating RMSE for test data to check accuracy
pred_test = LRMod.predict((test.iloc[:,0:9]).astype(float))
rmse_for_test =np.sqrt(mean_squared_error(test.iloc[:,9],pred_test))

print("Root Mean Squared Error For Test data = "+str(rmse_for_test))
print("R^2 Score(coefficient of determination) = "+str(r2_score(test.iloc[:,9],pred_test)))
#Root Mean Squared Error: 0.10079798693175389
# R^2 Score 0.0889341369651181

In [ ]:
num_df=df._get_numeric_data()
cols=list(set(df.columns) - set(num_df.columns))

In [ ]:
df_logit=pd.DataFrame()
df_logit=pd.get_dummies(data = df, columns = cols)

In [ ]:
cols = list(df_logit)
cols.insert(0, cols.pop(cols.index('Absenteeism_time_in_hours')))
df_logit = df_logit.ix[:, cols]


In [ ]:
train,test = train_test_split( df_logit, test_size = 0.20)
LRMod=sm.OLS(train.iloc[:,0],train.iloc[:,1:45]).fit()
LRMod.summary()

In [ ]:
pred_test = LRMod.predict((test.iloc[:,1:45]))
rmse_for_test =np.sqrt(mean_squared_error(test.iloc[:,0],pred_test))

print("Root Mean Squared Error For Test data = "+str(rmse_for_test))
print("R^2 Score(coefficient of determination) = "+str(r2_score(test.iloc[:,0],pred_test)))

# Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Building model on top of training dataset
fit_RF = RandomForestRegressor(n_estimators = 300).fit(train.iloc[:,0:9],train.iloc[:,9])


# Calculating RMSE for test data to check accuracy
pred_test = fit_RF.predict(test.iloc[:,0:9])
rmse_for_test =np.sqrt(mean_squared_error(test.iloc[:,9],pred_test))

print("Root Mean Squared Error For Test data = "+str(rmse_for_test))
print("R^2 Score(coefficient of determination) = "+str(r2_score(test.iloc[:,9],pred_test)))
# Root Mean Squared Error For Test data = 0.11950567245732827
# R^2 Score(coefficient of determination) = -0.021281881939122504

In [ ]:
df2.info()

In [ ]:
loss = df2[['Month of absence','Service time','Work load Average/day ','Absenteeism time in hours']]
loss["loss_month"] = (loss['Work load Average/day ']*loss['Absenteeism time in hours'])/loss['Service time']
loss["loss_month"] = np.round(loss["loss_month"]).astype('int64')
monthLoss={}
for i in range(1,13):
    monthLoss[i]=loss[loss['Month of absence'] == i]['loss_month'].sum()

In [ ]:
monthLoss